*****************************************************************
#  The Social Web 
- Instructors Davide Ceolin, Dayana Spagnuelo
- TA: Michael Accetto, Sarthak Gupta
- Exercises for Hands-on session 3
- 20.2.2020 11:00 - 12:45                 
- NU-6C-39, NU-6C-40                      
*****************************************************************

### Setup
Please first make sure your virtual environment is activated.

Prerequisites:
- Python 3.7
- virtual environment (recommended)

Software needed:
- Python packages: facebook-sdk, networkx, requests, ipython (which we'll install later in this notebook)

But let's check first if we're running a sufficiently new version of Python:

In [ ]:
import platform
import sys
# It's good practice to assert packages requirements at the beginning of a script:
assert sys.version_info >= (3, 6) # Tested with Python==3.7.5
# show python version
print("This jupyter notebook is running on Python " + platform.python_version())

Let's install what will be needed:

In [ ]:
!pip install networkx

To get the newer features available with the facebook-sdk package, we will install the package from its github repository. \
This is possible with pip, by following this syntax: \
`pip install git+YOUR_GITHUB_REPOSITORY_URL` \
For instance:

In [ ]:
!pip install -e git+https://github.com/mobolic/facebook-sdk.git#egg=facebook-sdk

If you haven't already requested a developer account, you can do so by following these steps:

- go to [developers.facebook.com](https://developers.facebook.com/)
- click on apps -> register as developer -> accept 
- go to: [https://developers.facebook.com/tools/explorer](https://developers.facebook.com/tools/explorer)
- select your app (create one if you don't see any)
- select 'user token'
- Add permissions under User Data Permission
- copy access token
- paste it in the cell below.

In [ ]:
import facebook
import json
# A helper function to pretty-print Python objects as JSON
p_top = lambda top: print("\n".join(["-"*25, top, "-"*25]))
                          
def pp(top, o):
    p_top()
    print(json.dumps(o, indent=1))

# Create a connection to the Graph API with your access token
token = 'EAAkbKojyZBAsBAJ4V4BnTxQfrdbEjUtEl5fEtvvq1jAvX7etT6IPQXaB60ZCwdj7RRa0xY3bwk4WanqsZBF4b00c1FIA3ifvnfJft8XLQPrG63kidkmlOUOP6FmoNj2hES3KiGdWiO5OcJUFGZCHdO14llGL0DFu5CZA47b7naeZCJm4SqJxLiULZCDc7JQCE0ZD'
g = facebook.GraphAPI(token)

# Execute a few sample queries

pp("Me", g.get_object('me'))
friends = g.get_connections(id='me', connection_name='friends')
pp("My friends", friends)
print()

If your friends list isn't publicly available, you should only be able to see your friends count.

In [ ]:

token = ""
app_graph = facebook.GraphAPI(access_token=token)
pp('Pages about UVA', app_graph.request('search', {'q' : 'Universiteit van Amsterdam', 'type' : 'page', 'limit' : 5}))

pp('Pages about VU', app_graph.request('search', {'q' : 'Vrije Universiteit Amsterdam', 'type' : 'page', 'limit' : 5}))

# Use the ids to query for likes
uva_id = '113928981951563'
vu_id = '116356121481'

# A quick way to format integers with commas every 3 digits
def int_format(n):
	return "{:,}".format(n)

print("UVA likes:", app_graph.get_object(uva_id)['likes'])
print("VU likes:", app_graph.get_object(vu_id)['likes'])

What Errors do you see? Are they a python problem or an API one?

******
Facebook is restricting automated access of information otherwise publicly available. 

However, keep in mind that you can still obtain interesting information from facebook, as long as you're willing to go through the appropriate [permission request procedure](https://developers.facebook.com/docs/facebook-login/permissions/). This entails building an app, submitting it to facebook and get its approval.

*******

Here some sources that can explain why facebook is filtering access to its resources, and that may inform your critical thinking.

- [Facebook relationships with EU's countries and politicians](https://www.theguardian.com/technology/2019/mar/02/facebook-global-lobbying-campaign-against-data-privacy-laws-investment)
- [Facebook relationships with other tech giants](https://www.nytimes.com/2018/12/18/technology/facebook-privacy.html)
- [Facebook's role in Brexit (Ted Talk by journalist Carole Cadwalladr)](https://www.ted.com/talks/carole_cadwalladr_facebook_s_role_in_brexit_and_the_threat_to_democracy)
- [Study on emotional contagion](https://www.pnas.org/content/111/24/8788.full) (in collaboration with facebook).
- [Facebook privacy policy](https://www.facebook.com/policy.php)

In order to continue our notebook, we'll download a dataset from:
https://snap.stanford.edu/data/egonets-Facebook.html

You can untar the package with `tar -xvzf facebook.tar.gz` \
If you use Windows, maybe [7zip](https://www.7-zip.org/download.html) can work for you.

Once downloaded and inflated, continue with the following steps:

In [ ]:
# list files in the ego-Facebook dataset (https://snap.stanford.edu/data/egonets-Facebook.html)
import os
from os import path
db_dir = "facebook" # replace with your download folder. 
db_relpaths = [path.join(db_dir, d) for d in os.listdir(db_dir)]
db_relpaths

In [ ]:
extensions = [f.split(".")[1] for f in db_relpaths] # trunk first part away
unique_extensions = sorted(set(extensions)) # create unique set
print(unique_extensions)

In [ ]:
# instead of declaring a list comprehension each time for each extension, we combine
# a lambda function with L.C. .
# Lambda function are declared in one line of code,
# take the argument before the column, and return what's after. 
ret_ext = lambda x: sorted([fn for fn in db_relpaths if fn.endswith(x)]

edges_fns = ret_ext("edges")
circles_fns = ret_ext("circles")
egofeat_fns = ret_ext("egofeat")
feat_fns = ret_ext(".feat") # The dot is important!
featnames_fns = ret_ext("featnames")

edges_fns 
#we sort for convenience, although alpha


Here each edge connects two nodes. we can show these edges by looking at the file:

In [ ]:
def show_dsf_head(filename):
    """Show head of given file for the ego-Facebook dataset."""
    p_top(filename.split(".")[1])
    with open(filename, "r") as e:
        [print(next(e)) for x in range(7)]
        
# show first 15 lines of edges.
show_dsf_head(edges_fns[0])

In [ ]:
show_head(circles_fns[0])
show_head(featnames_fns[0])
show_head(feat_fns[0])

Here encoded are a series of nodes, features and circles. 

Next: we will create a network x "Graph" object, and grow it by iterating through the edges files.

In [ ]:
import networkx as nx

G = nx.Graph() # here's our graph.
# Let's grow it!
for edge_file in edges_fns[0]:  
    # Once you've run the notebook, feel free to
    # remove the index "[0]" to experiment with the whole dataset.
    # For illustrative purposes we are only leaving the first bunch of connections.
    with open(edge_file, 'r') as f:
        for e in f:
            e1, e2 = e.strip().split(" ")
            G.add_edge(e1, e2)

In [ ]:
# we can see how nodes were automatically created by NX:
G.nodes()


In [ ]:
print("number of edges: " + str(G.number_of_edges()))
print("number of nodes: " + str(G.number_of_nodes()))

These are all the nodes and edges related to our "ego net" (i.e. a network of people on facebook).

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import rgb2hex
from matplotlib import cm
import numpy as np

rainbow_cmap = cm.get_cmap("gist_rainbow", G.number_of_nodes())

def color_breaks(cmap, n_breaks):
    """ Provides vector with n different hexadecimal values.
    Args:
        * cmap: a  matplotlib.Colormap object (https://matplotlib.org/3.1.3/tutorials/colors/colormaps.html)
        * n_breaks: number of discrete intervals to section the colormap
    Output:
        * n-long list of hexadecimal values.
    """
    breaks = np.linspace(0, 1, n_breaks)
    return [rgb2hex(cmap(bb)) for bb in breaks] 



options = {
    'node_color': color_breaks(rainbow_cmap, G.number_of_nodes()),
    'node_size': 5,
    'width': 1
}

In [ ]:
nx.draw(G, with_labels=False, **options)

In [ ]:
options['node_size'] = 1 # changing only one attribute it's easy with a dictionary!
nx.draw_shell(G, with_labels=False, **options)

And that's it!

As shown here, with NetworkX you can easily create networks and graph them, but also [add attributes to each node](https://networkx.github.io/documentation/stable/reference/functions.html#attributes).This can be used to the graph highlighting features or communities, according to the dimensions of your choice (e.g. such as political orientation, education and so on).